In [ ]:
class VanClient(object):
    def __init__(self, csv_file_path=None):
        self.van_path = '/home/erikadauria/.secret/van.json'
        self.driver = webdriver.Chrome()
        self.csv_file_path = csv_file_path
    def get_van_login(self):
        with open(self.van_path) as f:
            van_login = json.load(f)
            van_email = van_login['user_email']
            van_pw = van_login['password']
            van_pin = van_login['pin']
            van_twofa = van_login['twofa']
            return van_email, van_pw, van_pin, van_twofa
    def login_to_van(self):
        # Get Login Credentials
        user_email, pw, pin, twofa = self.get_van_login()
        # Go to VAN Login Page
        self.driver.get("https://www.votebuilder.com/Login.aspx?OIDF=1")
        # Select Login With Action ID
        self.driver.find_element_by_id("OpenIDSelectorLinkLink4").click()
        # Enter Login Info
        # Username
        username = self.driver.find_element_by_id("username")
        username.clear()
        username.send_keys(user_email)
        # Password
        password = self.driver.find_element_by_id("password")
        password.clear()
        password.send_keys(pw)
        # Select Login
        self.driver.find_element_by_xpath("//button[text()='Log In']").click()
        # Authentification Code
        self.complete_2fa(twofa)
        # Enter Login Pin
        #self.login_pin(pin)
        # Go to My Campaign
        self.driver.find_element_by_link_text("My Campaign").click()
    def complete_2fa(self, twofa):
        time.sleep(5)
        authcode = subprocess.check_output(['oathtool',  '-b', '--totp', str(twofa)])
        authcode = int(authcode)
        self.driver.find_element_by_id("Code").clear()
        self.driver.find_element_by_id("Code").send_keys(authcode)
        time.sleep(5)
        self.driver.find_element_by_id('sendBtn').click()
        time.sleep(3)
        try:
                elem = self.driver.find_element_by_xpath("//*[@id='body-content']//div[2]//h2[contains(text(),'Two-Factor Authentication')]")
                self.complete_2fa(twofa)
        except NoSuchElementException:
               pass   
    def login_pin(self, pin):
        xpaths = ["//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[4]/div[2]/div/span[2]",
                  "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[1]/div[1]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[1]/div[2]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[1]/div[3]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[2]/div[1]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[2]/div[2]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[2]/div[3]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[3]/div[1]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[3]/div[2]/div/span[2]",
                    "//*[@id='ctl00_ContentPlaceHolderVANPage_PanelWideColumn']/div/div[3]/div[3]/div/span[2]"]
        # Construct the letters corresponding to PIN
        letter_pin = ""
        for num in pin:
            letter_pin = letter_pin + str(self.driver.find_element_by_xpath(xpaths[int(num)]).text)
        # Enter the letters corresponding to PIN
        pin_pad = 'ctl00_ContentPlaceHolderVANPage_VANDetailsItemPIN_VANInputItemDetailsItemPINCode_PINCode'
        self.driver.find_element_by_id(pin_pad).send_keys(letter_pin)
        submit = 'ctl00_ContentPlaceHolderVANPage_ButtonNarrowSubmit'
        self.driver.find_element_by_id(submit).click()
    def turf_unturfed(self):
        #num_of_regions = self.get_csv_stats()
        # Log into VAN
        self.login_to_van()
        # Select Run Bulk Uploads
        # self.driver.find_element_by_id("ctl00_ContentPlaceHolderVANPage_ctl58_HyperLinkMenuBulkUpload").click()  -- Stopped working 10/2
        self.driver.find_element_by_id("ctl00_ContentPlaceHolderVANPage_ctl60_HyperLinkMenuBulkUpload").click()
        # Select Upload a New File
        self.driver.find_element_by_xpath("//*[@id='ctl00_ContentPlaceHolderVANPage_AccordionBulkUpload']/div[2]/div/table[1]/tbody/tr/td[2]/a").click()
        # Select Next
        self.driver.find_element_by_xpath("//*[@id='ctl00_ContentPlaceHolderVANPage_ButtonUploadSubmit']").click()
        # Select Direct Upload
        self.driver.find_element_by_xpath("//*[@id='ctl00_ContentPlaceHolderVANPage_VanDetailsItemFileSource_VANInputItemDetailsItemFileSource_FileSource_0']").click()
        # Select Select a File
        self.driver.find_element_by_id('ctl00_ContentPlaceHolderVANPage_InputFileDefault').send_keys(self.csv_file_path)
        # Select Upload
        self.driver.find_element_by_id('ctl00_ContentPlaceHolderVANPage_ButtonSubmitDefault').click()
        time.sleep(3)
        # In Case Unmatched Files
        self.driver.find_element_by_xpath("//input[@name = 'ctl00$ContentPlaceHolderVANPage$ctl06']").click()
        try:
            elem = self.driver.find_element_by_xpath("//*[@id='ctl00_ContentPlaceHolderVANPage_UnmatchedRecordsButtonPanel']/input[1]")
            self.driver.find_element_by_xpath("//*[@id='ctl00_ContentPlaceHolderVANPage_UnmatchedRecordsButtonPanel']/input[2]").click()
            self.driver.find_element_by_xpath("//input[@name = 'ctl00$ContentPlaceHolderVANPage$ctl06']").click()
        except NoSuchElementException:
            pass 
        # Select Apply Organizing Turfs for Apply Mapping
        time.sleep(5)
        self.driver.find_element_by_xpath("//select[@name='ctl00$ContentPlaceHolderVANPage$ctl03$AddULFieldID']/option[text()='Apply Organizing Turfs']").click()
        # Select Choose Column for Data Field for 2020 Coordinated Administrative Region
        time.sleep(3)
        self.driver.switch_to.frame(self.driver.find_element_by_name("RadWindow1"))
        self.driver.find_element_by_css_selector("input[type='radio'][value='Column']").click()
        # Select region_name for 2020 Coordinated Administrative Region
        self.driver.find_element_by_xpath("//select[@name='ctl01$ContentPlaceHolderVANPage$myLabelCont0$CF289$Column$ddl_Column']/option[text()='region_name']").click()
        # Select fo_name for 2020 Coordinated Administrative Organizer
        time.sleep(3)
        self.driver.find_element_by_xpath("//select[@name='ctl01$ContentPlaceHolderVANPage$myLabelCont0$CF290$Column$ddl_Column']/option[text()='fo_name']").click()
        # Select team_name for 2020 Coordinated Administrative Team
        time.sleep(3)
        self.driver.find_element_by_xpath("//select[@name='ctl01$ContentPlaceHolderVANPage$myLabelCont0$CF291$Column$ddl_Column']/option[text()='team_name']").click()
        # Select Next
        self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_Next0']").click()
        time.sleep(5)
        # Get the number of rows to be mapped from region_name to Activity Region
        row_count_region = len(self.driver.find_elements_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_1']/div/div[1]/table/tbody/tr"))
        for row in range(1, row_count_region): # Start at 1 to ignore the header row
            # Get text of region name from appropriate row (+1 because the rows are 2-indexed)
            region_name = self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_1']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[1]").text
            # Select dropdown of region_name in the corresponding row
            Select(self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_1']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[3]/span/select")).select_by_visible_text(region_name)
            time.sleep(.1)
        # Click next
        self.driver.find_element_by_id('ctl01_ContentPlaceHolderVANPage_Next1').click()
        # Get the number of rows to be mapped from fo_name to Activity Organizer
        time.sleep(5)
        row_count_foname = len(self.driver.find_elements_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_2']/div/div[1]/table/tbody/tr"))
        for row in range(1, row_count_foname): # Start at 1 to ignore the header row
            # Get text of fo_name from appropriate row (+1 because the rows are 2-indexed)
            fo_name = self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_2']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[2]").text
            # Select the dropdown of fo_name in the corresponding row
            self.driver.execute_script("return arguments[0].scrollIntoView();", self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_2']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[4]/span/select"))
            Select(self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_2']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[4]/span/select")).select_by_visible_text(fo_name)
            time.sleep(.1)
        time.sleep(5)
        self.driver.find_element_by_id('ctl01_ContentPlaceHolderVANPage_Next2').click()
        time.sleep(5)
        # Get the number of rows to be mapped from team_name to Activity Organizer
        row_count_teamname = len(self.driver.find_elements_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_3']/div/div[1]/table/tbody/tr"))
        for row in range(1, row_count_teamname): # Start at 1 to ignore the header row
            # Get text of team_name from appropriate row (+1 because the rows are 2-indexed)
            team_name = self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_3']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[3]").text
            # Select the dropdown of team_name in the corresponding row
            self.driver.execute_script("return arguments[0].scrollIntoView();", self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_3']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[5]/span/select"))
            Select(self.driver.find_element_by_xpath("//*[@id='ctl01_ContentPlaceHolderVANPage_3']/div/div[1]/table/tbody/tr["+str(row+1)+"]/td[5]/span/select")).select_by_visible_text(team_name)
            time.sleep(0.1)
        self.driver.find_element_by_id('ctl01_ContentPlaceHolderVANPage_Next3').click()
        time.sleep(5)
        self.driver.find_element_by_id('ctl00_ContentPlaceHolderVANPage_ButtonFinishUpload').click()
        time.sleep(5)
        self.driver.find_element_by_id('ctl00_ContentPlaceHolderVANPage_FinishUploadModal__submitButton').click()
        time.sleep(5)
